In [2]:
import pickle
import pandas as pd 
import numpy as np
import math
from statistics import median
import copy

In [3]:
with open('../data/trace_VecWordList.pkl',"rb") as f1, open('../data/trace_WordList.pkl', 'rb') as f2, open('../data/df_pre.pkl', 'rb') as f3:
    trace_VecWordList = pickle.load(f1) # Now holds vectors for each tokens in trace_VecWordList[trace][event][token].
    trace_WordList = pickle.load(f2) # Now holds the word tokens in same structure. Needed for idf calculation. 
    df = pickle.load(f3)

In [4]:
for i in df['event']:
    if 'sofortberweisung' in i:
        print(i)

In [29]:
np.multiply(2, trace_VecWordList[0][0])

array([[ 1.84845313e-01, -3.67387384e-03,  3.44955534e-01, ...,
        -5.75800091e-02, -7.14529082e-02,  1.99338898e-01],
       [ 7.23378547e-03, -1.46110542e-04,  2.21613217e-02, ...,
        -6.29764399e-04, -1.02698859e-02,  1.09480163e-02],
       [ 1.81732163e-01,  5.96556393e-03,  3.01808208e-01, ...,
        -4.65515293e-02, -6.23503886e-02,  1.82185069e-01],
       [ 1.90910697e-01, -1.25464015e-02,  3.33672673e-01, ...,
        -6.39402196e-02, -7.04220608e-02,  1.89847559e-01],
       [ 1.77107811e-01, -9.78758791e-04,  3.19016755e-01, ...,
        -4.29482721e-02, -7.95470029e-02,  1.76767692e-01]], dtype=float32)

In [22]:
#trace_WordList[3][5][1]
abs(-12)

12

## Bilden eines Trace Vektors mittels IDF
In dieser Methode werden die jeweiligen Vektoren gebildet, die die gesamten Traces repräsentieren. Für eine möglichst sinnvolle Gewichtung in diesem Prozess, wird die IDF (Inverse Document Frequency) verwendet. Die IDF ist eine in der NLP verbreitete Methoden und wird häufig in Zusammenhang mit der TF (Total Frequency) verwendet. Für unseren Fall eignet sich die IDF als alleiniges Merkmal besser. Die wichtige Funktion der IDF ist es, die Wörter entsprechend der "Wichtigkeit" zu bewerten. So werden werden Wörter, welche in vielen der Event vorkommen, eine geringere IDF aufweisen, als Wörter, die selten vertreten sind (Rechtschreibfehler oder seltene Zeitkategorisierungen)

Ein weiterer wichtiger Punkt, sind die Case IDs in der Methode. Da für den Usecase der Anomaly Detection in der Regel davon auszugehen ist, dass die Case IDs eindeutig und damit nicht von den Anomalies betroffen sind, werden die jeweiligen Vectoren nicht für den später gebildeten Trace Vektor berücksichtigt. Das Hinzuziehen von weiteren Vektoren, die keinen Wert für mögliche Anomalies liefern, wäre damit lediglich Noise. Die IDF der jeweiligen Case IDs gibt allerdingds darüber Aufschluss, wie lang oder kurz eine jeweilige Trace ist. Diese Tatsache machen wir uns zu nutze und nehmen die Länge der jeweiligen Traces in unsere Berechnung mit auf.

Inwieweit die Länge einer jeweiligen Trace eine Rolle spielt ist vermutlich je nach Log unterschiedlich. Es bleibt aber zu vermuten, dass die Länge der Traces in Event Logs typischerweise normalverteilt ist. Anomalies entstehen bei dieser Verteilung daher an den äußeren Rändern. In unserer Berechnung bilden wir den Median der für die Case IDs berechneten IDF-Werte. Dieser gibt uns Aufschluss über die "typische" Länge der Traces. Hierbei wurde sicht bewusst gegen den Durschnitt entschieden, um den Einfluss möglicher Outlier gering zu halten. Später multiplizieren wir die einzelnen Vektoren der Traces noch mit der jeweiligen Abweichung vom Median, um den Abstand vom Median Wert zu verdeutlichen. Die Werte liegen hier in [1, 1.31] und haben damit erst bei großer Abweichung eine Auswirkung auf die Ergebnisse.

In [39]:
def build_traceVector(df, traceVectorList, traceWordList):
    """Method to process data created by word model. It will form a summed up vector for each trace and weigh in tokens with calculated IDF."""
    
    unique_tokens = set() # Will hold unique values for all columns. 
    
    unique_tokens.update(list(df['timestamp_diff'].map(str).unique()))
    unique_tokens.update(list(df['id'].map(str).unique()))
    unique_tokens.update(list(df['event'].str.replace('[^a-zA-Z ]', '').str.lower().str.split(' ').sum()))
    
    idf_dict = dict.fromkeys(unique_tokens, 0) # Initiate dictionary holding all unique keys with 0 as values. 
    
    #Building up dictionary and upping the counter for every event which contains the word.
    for trace in traceWordList:
        for event in trace:
            for uniqueToken in unique_tokens:
                if uniqueToken in event:
                    idf_dict[uniqueToken] += 1
    
    # Calculating IDF with log of basis 10.
    event_len = len(df)
    for key in idf_dict:
        if idf_dict[key] != 0:
            idf_dict[key] = math.log10(event_len/(idf_dict[key]+1))
    
    # Building up list of IDF values for Case IDs and calculating the median. 
    unique_IDs = df['id'].unique() # automatically sorted 
    median_list = []
    
    for caseID in unique_IDs: 
        median_list.append(idf_dict[str(caseID)])
    median_case = median(median_list) # Represents the median idf score for the case IDs.
    
    # Update traceVectorList by multiplying items with the idf score. Case ID will be multiplied by idf-score/median_case.
    idfVectorList = copy.deepcopy(traceVectorList)
    for traceID in range(len(traceVectorList)): #Representing array index and not real trace IDs
        for eventID in range(len(traceVectorList[traceID])):
            event_list = [] # List in which the multiplied vectors of each event token are stored.
            for tokenID in range(len(traceVectorList[traceID][eventID])):
                
                if tokenID != 1: # Ignore case IDs
                    idf_val = idf_dict[traceWordList[traceID][eventID][tokenID]] # Holding idf value of current token.
                    word_vec = traceVectorList[traceID][eventID][tokenID] # The vector of current token.
                    multiplied_Vec = np.multiply(idf_val,word_vec)
                    event_list.append(multiplied_Vec)
                
            idfVectorList[traceID][eventID] = event_list # Add list of vectors to new list of multiplied values.
    
    # Instead of factoring in vector of case IDs we will take the trace length into account. It is more likely to have anomalies in traced with high deviation from median length.
    # After that we will sum up the event vectors and then trace vectors in the next step.
    idfSumList = copy.deepcopy(idfVectorList) # Will hold the summed up trace vectors.
    for traceIndex in range(len(idfVectorList)):
        median_difference = abs(median_case-idf_dict[str(unique_IDs[traceIndex])])+1 # Median minus the IDF of each case ID. Works because unique_IDs and traceVectorList are sorted..
        print(f"Is multiplied by {median_difference}")
        for eventIndex in range(len(idfVectorList[traceIndex])):
            idf_EventList = np.multiply(median_difference,idfVectorList[traceIndex][eventIndex]) # Multiplying all vectos by the median difference.
            idfSumList[traceIndex][eventIndex] = np.add.reduce(idf_EventList) # Building the sum vector of the event vectors.
            
        idfSumList[traceIndex] = np.add.reduce(idfSumList[traceIndex])
            
    return idfSumList

In [40]:
returnList = build_traceVector(df, trace_VecWordList, trace_WordList)

C:\Users\nbloe\AppData\Local\Temp\ipykernel_10952\1576924413.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  unique_tokens.update(list(df['event'].str.replace('[^a-zA-Z ]', '').str.lower().str.split(' ').sum()))


Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0579919469776868
Is multiplied by 1.0969100130080567
Is multiplied by 1.0579919469776868
Is multiplied by 1.0579919469776868
Is multiplied by 1.2430380486862944
Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0
Is multiplied by 1.0
Is multiplied by 1.0511525224473814
Is multiplied by 1.0
Is multiplied by 1.0511525224473814
Is multiplied by 1.0579919469776868
Is multiplied by 1.0
Is multiplied by 1.0
Is multiplied by 1.0
Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0579919469776868
Is multiplied by 1.0511525224473814
Is multiplied by 1.3010299956639813
Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0579919469776868
Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0579919469776868
Is multiplied by 1.0
Is multiplied by 1.0
Is multiplied by 1.057

In [37]:
returnList[0]

array([ 2.35065   , -0.17266162,  4.279239  , -0.8910073 , -4.3006806 ,
       -4.847078  , -2.2499344 ,  7.4515076 , -3.6932645 ,  0.21293673,
       -1.6189204 , -1.0027478 , -0.12077584,  2.120418  ,  0.5029834 ,
       -0.11837414, -3.7459917 ,  3.2526054 , -3.461852  , -0.68506116,
       -1.8001064 ,  3.7080324 ,  4.148725  , -3.209156  , -1.5063729 ,
        0.923847  ,  1.9316978 , -1.1069034 , -5.1679916 ,  0.9040306 ,
       -0.41679046,  0.33144587, -0.34235516, -1.7003632 ,  3.0992064 ,
        1.1644237 ,  6.387888  , -2.233181  ,  1.4545295 , -1.4855608 ,
       -2.5626736 ,  1.4052454 , -2.039855  , -1.8620279 ,  0.2940551 ,
       -1.1999437 ,  0.6261321 ,  0.60757816,  3.7792628 , -2.3768678 ,
       -4.9924498 ,  1.886135  , -1.8470061 , -2.351036  ,  7.669965  ,
       -1.9351451 ,  3.1952076 ,  0.5516548 , -3.599975  , -4.402649  ,
        2.1878057 , -3.2459502 ,  1.6862636 ,  0.8947018 ,  3.535439  ,
        5.150761  , -0.18420297,  1.870139  , -1.0110964 , -0.18

In [10]:
trace_VecWordList[0][0][0]

array([ 0.09242266, -0.00183694,  0.17247777, -0.02903248, -0.1738679 ,
       -0.19111857, -0.09224577,  0.2958652 , -0.14744213,  0.01377219,
       -0.06588571, -0.0379857 , -0.00852994,  0.08716642,  0.01821577,
       -0.00432784, -0.1538048 ,  0.13143806, -0.13576885, -0.02683246,
       -0.07510457,  0.14984712,  0.15655991, -0.13041185, -0.06068774,
        0.03245863,  0.07291745, -0.03694805, -0.2036992 ,  0.03383278,
       -0.01856805,  0.01034486, -0.014867  , -0.06507783,  0.11989424,
        0.05170462,  0.25608024, -0.09649123,  0.04941764, -0.05967883,
       -0.09970379,  0.05088067, -0.0828876 , -0.06901053,  0.01482841,
       -0.04529485,  0.01742649,  0.01547979,  0.14615609, -0.10050087,
       -0.1974544 ,  0.07279113, -0.07376505, -0.09246977,  0.30336994,
       -0.07569245,  0.12810428,  0.02243096, -0.13762006, -0.17279787,
        0.08746767, -0.1302357 ,  0.0633596 ,  0.03416296,  0.14185311,
        0.20375672, -0.00755817,  0.07597748, -0.03808793, -0.00

In [11]:
returnList[0][0][0]

array([ 0.07461028, -0.00148291,  0.13923658, -0.02343712, -0.14035879,
       -0.15428479, -0.07446748,  0.23884387, -0.119026  ,  0.01111791,
       -0.05318773, -0.03066482, -0.00688599,  0.07036707,  0.01470509,
       -0.00349375, -0.1241624 ,  0.10610635, -0.10960247, -0.02166111,
       -0.06062986,  0.12096748,  0.12638654, -0.10527791, -0.04899155,
        0.02620297,  0.05886426, -0.02982715, -0.16444078,  0.02731228,
       -0.01498948,  0.00835112, -0.01200172, -0.05253555,  0.09678734,
        0.04173972,  0.20672657, -0.07789473,  0.03989351, -0.04817708,
       -0.08048814,  0.04107457, -0.0669129 , -0.05571031,  0.01197057,
       -0.0365653 ,  0.01406793,  0.01249641,  0.1179878 , -0.0811316 ,
       -0.15939952,  0.05876229, -0.0595485 , -0.07464831,  0.24490224,
       -0.06110444,  0.10341508,  0.0181079 , -0.1110969 , -0.13949499,
        0.07061026, -0.10513572,  0.05114847,  0.02757882,  0.11451413,
        0.16448721, -0.0061015 ,  0.06133454, -0.03074734, -0.00